In [2]:
#Importation des modules 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from pandas_datareader.data import DataReader
from pathlib import Path
import requests, io

In [3]:
#lecture des rendements mensuels
# index --> colonne date devient l'index
# parse --> tranforme la date en objet date
returns = pd.read_csv("../data/returns_monthly.csv", index_col=0, parse_dates=True)

print("Dimensions :", returns.shape)
returns.head()

Dimensions : (126, 3)


,VFV,XIU,ZAG
Date,,,
2015-02-28,0.041121,0.038285,-0.001744
2015-03-31,-0.003815,-0.020445,-0.003590
2015-04-30,-0.038967,0.020548,-0.012792
2015-05-31,0.045027,-0.010738,0.000710
2015-06-30,-0.015795,-0.029211,-0.004891


In [4]:
#On observe rapidement à quoi ressemble la série pour corriger le code de la suite 
import requests, textwrap

url_dbg = (
    "https://www.bankofcanada.ca/valet/observations/group/"
    "tbill_wednesday/csv?start_date=2025-01-01&end_date=2025-01-31"
)

raw_lines = requests.get(url_dbg, timeout=10).text.splitlines()

# Affiche les 40 premières lignes avec numéros
for i, ln in enumerate(raw_lines[:40]):
    print(f"{i:02d}: {ln}")

00: ﻿"TERMS AND CONDITIONS"
01: "https://www.bankofcanada.ca/terms/"
02: 
03: "NAME"
04: "Selected Treasury Bill Yields - Weekly: Wednesday"
05: 
06: "DESCRIPTION"
07: "1, 3, 6 and 12 month Treasury bill data."
08: 
09: "LINK"
10: "https://www.bankofcanada.ca/?p=39889"
11: 
12: "SERIES"
13: "id","label","description"
14: "V80691342","1 month","Treasury bills"
15: "V80691344","3 month","Treasury bills"
16: "V80691345","6 month","Treasury bills"
17: "V80691346","1 year","Treasury bills"
18: 
19: "OBSERVATIONS"
20: "date","V80691342","V80691344","V80691345","V80691346"
21: "2025-01-01","3.19","3.16","3.09","3.03"
22: "2025-01-08","3.14","3.11","3.06","3.01"
23: "2025-01-15","3.14","3.10","3.08","3.09"
24: "2025-01-22","3.09","3.03","2.98","3.01"
25: "2025-01-29","2.99","2.89","2.90","2.89"


In [5]:
# ── 1. Période alignée sur 'returns' ────────────────────────────────────
start = returns.index[0].strftime("%Y-%m-%d")
end   = returns.index[-1].strftime("%Y-%m-%d")

url = ( "https://www.bankofcanada.ca/valet/observations/group/"
        f"tbill_wednesday/csv?start_date={start}&end_date={end}" )

csv_lines = requests.get(url, timeout=10).text.splitlines()

# ── 2. Trouver la vraie ligne d'en-tête  ("date", "V80691344", …) ───────
for i, ln in enumerate(csv_lines):
    if ln.lower().startswith('"date"') and "v80691344" in ln.lower():
        header_idx = i
        break
else:
    raise RuntimeError("En-tête contenant V80691344 introuvable.")

data_str = "\n".join(csv_lines[header_idx:])   # header + observations

# ── 3. Lecture pandas  ---------------------------------------------------
df = pd.read_csv(io.StringIO(data_str))

# Colonnes : date, V80691342, V80691344, …
df.columns = [c.strip().lower() for c in df.columns]
rf_daily = (df[["date", "v80691344"]]
              .rename(columns={"v80691344": "yield"}))
rf_daily["date"] = pd.to_datetime(rf_daily["date"])
rf_daily["yield"] = rf_daily["yield"].astype(float)
rf_daily = rf_daily.set_index("date").sort_index()

# ── 4. Fin de mois  →  rendement mensuel décimal ------------------------
rf_monthly = (
    rf_daily["yield"]
      .resample("ME").last()
      .ffill()
      .div(100)          # 4.55 % → 0.0455 (annuel)
      .div(12)           # annuel → mensuel décimal
      .rename("RF")
)

display(rf_monthly.head())# contrôle visuel
rf_monthly.to_csv("../data/rf_monthly.csv")


date
2015-03-31    0.000450
2015-04-30    0.000550
2015-05-31    0.000525
2015-06-30    0.000483
2015-07-31    0.000342
Freq: ME, Name: RF, dtype: float64

In [6]:
# calcul du sigma et ratio de Sharpe

ANN = 12
excess = returns.sub(rf_monthly, axis=0)

stats = pd.DataFrame({"Annual Return %": returns.mean()*ANN*100, "Volatility %": returns.std()*np.sqrt(ANN)*100, "Sharpe": excess.mean()/returns.std()*np.sqrt(ANN)}).round(2)

display(stats)
stats.to_csv("../data/perf_stats.csv")

,Annual Return %,Volatility %,Sharpe
VFV,13.93,13.05,0.91
XIU,9.73,12.41,0.62
ZAG,1.44,5.29,-0.05
